<a href="https://colab.research.google.com/github/GoodJinMo/kfq_ML_2023/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **파이썬을 활용한 머신러닝 응용**

---

http://scikit-learn.org



In [32]:
import sklearn


In [ ]:
dir('sklearn')

In [ ]:
help('sklearn')

In [35]:
print(sklearn.__version__)


1.2.2


In [63]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score  #정확도 점수
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


# iris

In [31]:
iris = load_iris()
print(f"키 목록 :{iris.keys()}",
      f"featrue_names의타입 :{type(iris.feature_names)}",
      f"featrue_names의길이 :{len(iris.feature_names)}",
      f"featrue_names:{iris.feature_names}",sep="\n")

키 목록 :dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
featrue_names의타입 :<class 'list'>
featrue_names의길이 :4
featrue_names:['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [23]:
iris.feature_names #columns name
iris.data #독립변수
iris.target #종속변수

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['label']=iris.target
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [7]:
x_train,x_test, y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.2)
x_train.shape, x_test.shape,y_train.shape,y_test.shape

#train_test_split(arrays, test_size, train_size, random_state, shuffle, stratify)
#

((120, 4), (30, 4), (120,), (30,))

In [30]:
dt_cf=DecisionTreeClassifier() #모델생성
dt_cf.fit(iris.data,iris.target) #학습
pred=dt_cf.predict(x_test) #예측
print(pred,y_test,sep='\n')
print(f"accuracy:{accuracy_score(pred,y_test):.4f}")


[1 2 2 0 0 0 2 0 2 0 1 0 2 0 2 1 1 1 2 0 2 0 1 2 1 1 2 0 1 2]
[1 2 2 0 0 0 2 0 2 0 1 0 2 0 2 1 1 1 2 0 2 0 1 2 1 1 2 0 1 2]
accuracy:1.0000


In [29]:
dt_cf=DecisionTreeClassifier() #모델생성
dt_cf.fit(x_train,y_train) #학습
pred1=dt_cf.predict(x_test) #예측
print(pred1,y_test,sep='\n')
print(f"accuracy:{accuracy_score(pred1,y_test):.4f}")

[1 2 2 0 0 0 2 0 2 0 1 0 2 0 2 1 1 1 2 0 2 0 2 2 1 1 2 0 1 2]
[1 2 2 0 0 0 2 0 2 0 1 0 2 0 2 1 1 1 2 0 2 0 1 2 1 1 2 0 1 2]
accuracy:0.9667


# k폴드 교차 검증

In [38]:
#k폴드 교차 검증
#k개의 데이터 폴드 세트를 만들어서 k번만큼 각 폴트 세트에 학습과 검증 평가를 반복 수행
iris = load_iris()
features = iris.data
label = iris.target
print("세트 크기:",features.shape[0])


세트 크기: 150


In [64]:
dt_clf = DecisionTreeClassifier() #모델 생성
kfold = KFold(n_splits=5)
cv_accuracy =[]
n_iter = 0
for train, test in kfold.split(features):
    x_train,x_test = features[train], features[test]
    y_train,y_test = label[train], label[test]

    dt_clf.fit(x_train,y_train) #학습
    pred = dt_clf.predict(x_test) #예측
    n_iter+=1

    acc = np.round(accuracy_score(pred,y_test),3)
    print(n_iter,"회 교차 검증 accuracy:",acc)
    cv_accuracy.append(acc)
print("평균 accuracy:",np.mean(cv_accuracy))

1 회 교차 검증 accuracy: 1.0
2 회 교차 검증 accuracy: 0.967
3 회 교차 검증 accuracy: 0.867
4 회 교차 검증 accuracy: 0.933
5 회 교차 검증 accuracy: 0.733
평균 accuracy: 0.9
0    50
1    50
2    50
Name: label, dtype: int64


In [65]:
print(df['label'].value_counts())

0    50
1    50
2    50
Name: label, dtype: int64


In [62]:
kfold = KFold(n_splits=3)
xdf=df.drop('label',axis=1)
features=iris.data
label = iris.target
n = 0
for train,test in kfold.split(xdf):
    n+=1
    label_train = df['label'].iloc[train]
    label_test = df['label'].iloc[test]

    x_train,x_test = features[train], features[test]
    y_train,y_test = label[train], label[test]

    dt_clf.fit(x_train,y_train) #학습
    pred = dt_clf.predict(x_test) #예측
    acc = np.round(accuracy_score(pred,y_test),3)
    cv_accuracy.append(acc)

    print("##",n,"회 교차 검증",'##')
    print("학습 데이터 \n",label_train.value_counts(),'\n')
    print("검증 데이터 \n",label_test.value_counts(),"\n")
    print(n,"회 교차 검증 accuracy:",acc)
    print('---------------------------------------------------',"\n")
print("평균 accuracy:",np.mean(cv_accuracy))

## 1 회 교차 검증 ##
학습 데이터 
 1    50
2    50
Name: label, dtype: int64 

검증 데이터 
 0    50
Name: label, dtype: int64 

1 회 교차 검증 accuracy: 0.0
--------------------------------------------------- 

## 2 회 교차 검증 ##
학습 데이터 
 0    50
2    50
Name: label, dtype: int64 

검증 데이터 
 1    50
Name: label, dtype: int64 

2 회 교차 검증 accuracy: 0.0
--------------------------------------------------- 

## 3 회 교차 검증 ##
학습 데이터 
 0    50
1    50
Name: label, dtype: int64 

검증 데이터 
 2    50
Name: label, dtype: int64 

3 회 교차 검증 accuracy: 0.0
--------------------------------------------------- 

평균 accuracy: 0.0


In [70]:
kfold = StratifiedKFold(n_splits=3)
features=iris.data
label = iris.target
cv_accuracy =[]

n = 0
for train,test in kfold.split(xdf,df['label']):
    n+=1
    label_train = df['label'].iloc[train]
    label_test = df['label'].iloc[test]

    x_train,x_test = features[train], features[test]
    y_train,y_test = label[train], label[test]

    dt_clf.fit(x_train,y_train) #학습
    pred = dt_clf.predict(x_test) #예측
    acc = np.round(accuracy_score(pred,y_test),3)
    cv_accuracy.append(acc)

    print("##",n,"회 교차 검증",'##')
    print("학습 데이터 \n",label_train.value_counts(),'\n')
    print("검증 데이터 \n",label_test.value_counts(),"\n")
    print(n,"회 교차 검증 accuracy:",acc)
    print('---------------------------------------------------',"\n")
print("평균 accuracy:",np.mean(cv_accuracy))

## 1 회 교차 검증 ##
학습 데이터 
 2    34
0    33
1    33
Name: label, dtype: int64 

검증 데이터 
 0    17
1    17
2    16
Name: label, dtype: int64 

1 회 교차 검증 accuracy: 0.98
--------------------------------------------------- 

## 2 회 교차 검증 ##
학습 데이터 
 1    34
0    33
2    33
Name: label, dtype: int64 

검증 데이터 
 0    17
2    17
1    16
Name: label, dtype: int64 

2 회 교차 검증 accuracy: 0.92
--------------------------------------------------- 

## 3 회 교차 검증 ##
학습 데이터 
 0    34
1    33
2    33
Name: label, dtype: int64 

검증 데이터 
 1    17
2    17
0    16
Name: label, dtype: int64 

3 회 교차 검증 accuracy: 1.0
--------------------------------------------------- 

평균 accuracy: 0.9666666666666667


# GridSearchCv
---
교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에


In [92]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.model_selection import GridSearchCV


In [93]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()

data = iris.data
label = iris.target

scores = cross_val_score(dt_clf,data,label,scoring='accuracy',cv=3)
print("교차 검증 accuracy:",np.round(scores,4))
print("평균 검증 accuracy:",np.round(np.mean(scores),4))


교차 검증 accuracy: [0.98 0.94 1.  ]
평균 검증 accuracy: 0.9733


In [ ]:
dt=DecisionTreeClassifier()
scores=cross_validate(dt,data,label,
                      scoring=['accuracy','precision','recall'],
                      cv=3)
scores

In [95]:
iris = load_iris()
x_train, x_test, y_train,y_test = train_test_split(iris.data,iris.target, test_size=0.2)

mds=[1,2,3]
mss=[2,3,4]
for depth in mds:
  for ms in mss:
    dt=DecisionTreeClassifier(max_depth=depth,min_samples_split=ms)
    dt.fit(x_train,y_train)
    pred=dt.predict(x_test)
    print(depth,",",ms, ": acc=",accuracy_score(y_test,pred),"\n")

1 , 2 : acc= 0.5666666666666667 

1 , 3 : acc= 0.5666666666666667 

1 , 4 : acc= 0.5666666666666667 

2 , 2 : acc= 0.9333333333333333 

2 , 3 : acc= 0.9333333333333333 

2 , 4 : acc= 0.9333333333333333 

3 , 2 : acc= 0.9666666666666667 

3 , 3 : acc= 0.9666666666666667 

3 , 4 : acc= 0.9666666666666667 



In [96]:
parameters = {'max_depth':[1,2,3],
              'min_samples_split':[2,3]
              }


In [97]:
dt=DecisionTreeClassifier() #모델 두두둥장
grid_dt=GridSearchCV(dt,param_grid=parameters,cv=3,refit=True)
grid_dt.fit(x_train,y_train) #학습 뚜잉

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [98]:
print("최적 파라미터:",grid_dt.best_params_)
print("최고 accuracy:",grid_dt.best_score_)


최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
최고 accuracy: 0.9416666666666668


## 전처리

---


In [107]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [100]:
items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print("after:",labels)


after: [0 1 4 5 3 3 2 2]


In [106]:
items=np.array(items).reshape(-1,1)

In [112]:
ohe=OneHotEncoder()
ohe.fit(items)
labels=ohe.transform(items)
labels.toarray()


array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [114]:
items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','믹서','믹서']
df=pd.get_dummies(items)
df

,TV,냉장고,믹서,선풍기,전자레인지,컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


In [120]:
from scipy import stats

In [122]:
heights = stats.norm.rvs(loc=173,scale=4,size=1000)
bmis = stats.norm.rvs(loc=21,scale=2,size=1000)
weights = bmis*(heights**2)/10000

In [123]:
c_sizes = [ (heights[i]*2+weights[i]*2)/5  for i in range(1000)]
c_sizes = np.array(c_sizes)

In [130]:
from sklearn.neighbors import KNeighborsRegressor
df = pd.DataFrame()
df['height'] = heights
df['weight'] = weights
data = df.values
label = c_sizes

In [131]:
x_train, x_test, y_train, y_test = train_test_split(data,label)

In [151]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)
knn.score(x_test,y_test)

0.9945834861763081

In [158]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
print(df.mean(),df.var(),sep='\n\n')

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [161]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)
scaled = scaler.transform(df)
df2 = pd.DataFrame(scaled)
df2.de


,0,1,2,3
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444
...,...,...,...,...
145,1.038005,-0.131979,0.819596,1.448832
146,0.553333,-1.282963,0.705921,0.922303
147,0.795669,-0.131979,0.819596,1.053935
148,0.432165,0.788808,0.933271,1.448832


In [162]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(df)
scaled = mms.transform(df)
df2 = pd.DataFrame(scaled)
df2.describe()




,0,1,2,3
count,150.000000,150.000000,150.000000,150.000000
mean,0.428704,0.440556,0.467458,0.458056
std,0.230018,0.181611,0.299203,0.317599
min,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.333333,0.101695,0.083333
50%,0.416667,0.416667,0.567797,0.500000
75%,0.583333,0.541667,0.694915,0.708333
max,1.000000,1.000000,1.000000,1.000000


In [163]:
mms = MinMaxScaler()
mms.fit(data)
scaled_data = mms.transform(data)

x_train, x_test, y_train, y_test = train_test_split(scaled_data,label)
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)
knn.score(x_test,y_test)

0.9968702064355502

In [164]:
x_train, x_test, y_train, y_test = train_test_split(data,label)
mms = MinMaxScaler()
mms.fit(x_train)
scaled_train = mms.transform(x_train)
mms = MinMaxScaler()
mms.fit(x_test)
scaled_test = mms.transform(x_test)


In [166]:
sample = [[1.75,89],[1.72,62]]
mms.transform(sample)

array([[-5.99998172,  1.18841589],
       [-6.00112688,  0.44294669]])